In [ ]:
# Following is the tutorial on how to create and deploy a Matching Engine index which will store your data & embeddings
# After you deploy the index, you will be able to query it for semantically similar data, to, for example, get answers for a
# question, like it was shown on the ChatDKG Office Hours #5

In [ ]:
from google.colab import auth
auth.authenticate_user() # Authenticate first so you can use the Google Cloud commands

PROJECT_ID = "ot-demo"

In [ ]:
!gcloud config set project "ot-demo"

In [ ]:
!pip install google-cloud-aiplatform==1.21.0

In [ ]:
from google.cloud import aiplatform
from google.protobuf import struct_pb2
import time

In [ ]:
aiplatform.init(project="ot-demo", location="us-central1")

In [ ]:
REGION = "us-central1"
ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

PARENT = "projects/{}/locations/{}".format(PROJECT_ID, REGION)

PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]
print(PROJECT_NUMBER)

In [ ]:
NETWORK_NAME = "ot-vpc-network" # This name will be visible in the interface
PEERING_RANGE_NAME = "ot-range" # This name will be visible in the interface

# Below is the setup to create a VPC network for your Google Vertex AI project, which is needed to deploy the index

In [ ]:
! gcloud compute networks create {NETWORK_NAME} --bgp-routing-mode=regional

In [ ]:
! gcloud compute firewall-rules create {NETWORK_NAME}-allow-icmp --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow icmp
! gcloud compute firewall-rules create {NETWORK_NAME}-allow-internal --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow all --source-ranges 10.128.0.0/9
! gcloud compute firewall-rules create {NETWORK_NAME}-allow-rdp --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow tcp:3389
! gcloud compute firewall-rules create {NETWORK_NAME}-allow-ssh --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow tcp:22

In [ ]:
! gcloud compute addresses create {PEERING_RANGE_NAME} --global --prefix-length=16 --network={NETWORK_NAME} --purpose=VPC_PEERING --project={PROJECT_ID} --description="peering range for uCAIP Haystack."

In [ ]:
!gcloud services enable servicenetworking.googleapis.com --project="ot-demo"

In [ ]:
! gcloud services vpc-peerings connect --service=servicenetworking.googleapis.com --network={NETWORK_NAME} --ranges={PEERING_RANGE_NAME}

In [ ]:
# Create a Google Cloud bucket: https://console.cloud.google.com/storage/browser in the same!!! region as your project
# Copy your data.json from the notebook to the bucket, this can also be done from the interface
# Data.json as of writing of this comment should be in the following format:
# Every line should contain a valid JSON object with the properties 'id' with a string value and 'embedding' with an array value
! gsutil cp data.json gs://ot-demox/matching-engine/data/text/data.json

In [ ]:
index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name="ot_index_display_name",
    contents_delta_uri="gs://ot-demo/matching-engine/data/text/", # path to the data on the bucket
    dimensions=768, # dimensions of the vector, make sure this matches the embedding model dimension you use
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE", # distance function your model uses, check this on huggingfaces for example
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=7,
    description="description here",
    labels={"label_name": "label_value"},
)

In [ ]:
# check whether your index was created via this command or the interface
!gcloud ai indexes list \
  --project="ot-demo" \
  --region="us-central1"